In [14]:
#
# IPython magic functions to use with Pyspark and Spark SQL
# The following code is intended as examples of shorcuts to simplify the use of SQL in pyspark
# The defined functions are:
#
# %sql <statement>          - return a Spark DataFrame for lazy evaluation of the SQL
# %sql_show <statement>     - run the SQL statement and show max_show_lines (50) lines
# %sql_display <statement>  - run the SQL statement and display the results using a HTML table
#                           - this is implemented passing via Pandas and displays up to max_show_lines (50)
# %sql_explain <statement>  - display the execution plan of the SQL statement
#
# Use: %<magic> for line magic or %%<magic> for cell magic.
#
# Author: Luca.Canali@cern.ch
# September 2016
#

import findspark

findspark.init()

import pyspark
import random
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
from pyspark.sql import SQLContext

from src.database import UrbsNeo4JDatabase

conf = SparkConf().setAppName("App")
conf = (conf.setMaster('local[*]')
        .set('spark.executor.memory', '4G')
        .set('spark.driver.memory', '30G')
        .set('spark.driver.maxResultSize', '10G'))

sc = SparkContext.getOrCreate(conf=conf)
sqlContext = SQLContext(sc)


from IPython.core.magic import register_line_cell_magic

# Configuration parameters
max_show_lines = 50         # Limit on the number of lines to show with %sql_show and %sql_display
detailed_explain = True     # Set to False if you want to see only the physical plan when running explain


@register_line_cell_magic
def sql(line, cell=None):
    "Return a Spark DataFrame for lazy evaluation of the sql. Use: %sql or %%sql"
    val = cell if cell is not None else line 
    return sqlContext.sql(val)

@register_line_cell_magic
def sql_show(line, cell=None):
    "Execute sql and show the first max_show_lines lines. Use: %sql_show or %%sql_show"
    val = cell if cell is not None else line 
    
    return sqlContext.sql(val).show(max_show_lines) 

@register_line_cell_magic
def sql_display(line, cell=None):
    """Execute sql and convert results to Pandas DataFrame for pretty display or further processing.
    Use: %sql_display or %%sql_display"""
    val = cell if cell is not None else line 
    return sqlContext.sql(val).limit(max_show_lines).toPandas() 

@register_line_cell_magic
def sql_explain(line, cell=None):
    "Display the execution plan of the sql. Use: %sql_explain or %%sql_explain"
    val = cell if cell is not None else line 
    return sqlContext.sql(val).explain(detailed_explain)

In [16]:
processed_path = '/home/altieris/datascience/data/curitibaurbs/processed/'

linhas = sqlContext.read.json(processed_path+'linhas/')
linhas.registerTempTable("linhas")

pontosLinha = sqlContext.read.json(processed_path+'pontoslinha/')
pontosLinha.registerTempTable("pontos_linha")


In [18]:
 query_view_rota_sequenciada = "CREATE OR REPLACE TEMPORARY VIEW rota_sequenciada AS  " \
                                  "select 	pseq.cod_linha,pseq.sentido_linha,pseq.seq_inicio,pseq.seq_fim,pseq.ponto_inicio,pseq.nome_ponto_inicio " \
                                  ",pseq.ponto_final,pseq.nome_ponto_final,li.CATEGORIA_SERVICO as categoria_servico,li.NOME as nome_linha,li.NOME_COR as nome_cor,li.SOMENTE_CARTAO as somente_cartao " \
                                  ",pseq.sourcedate " \
                                  "from (select " \
                                  "p1.COD as cod_linha " \
                                  ",p1.SENTIDO  as sentido_linha " \
                                  ",p1.SEQ      as seq_inicio " \
                                  ",p2.SEQ      as seq_fim " \
                                  ",p1.NUM      as ponto_inicio " \
                                  ",p1.NOME     as nome_ponto_inicio " \
                                  ",p2.NUM      as ponto_final " \
                                  ",p2.NOME     as nome_ponto_final " \
                                  ",p1.sourcedate " \
                                  "from pontos_linha P1 " \
                                  "inner join pontos_linha p2 on (p1.SEQ+1 = p2.SEQ and p1.COD = p2.COD and p1.SENTIDO = p2.SENTIDO and p1.sourcedate = p2.sourcedate) " \
                                  ") pseq " \
                                  "inner join linhas       li on (pseq.cod_linha = li.COD and pseq.sourcedate = li.sourcedate) " \
                                  "order by pseq.cod_linha,pseq.sentido_linha,pseq.seq_inicio,pseq.seq_fim "

sqlContext.sql(query_view_rota_sequenciada)

DataFrame[]

In [22]:
query_rotas = "select cod_linha            as line_code" \
      ",sentido_linha        as line_direction " \
      ",ponto_inicio         as start_point " \
      ",seq_inicio           as sequence_number_start " \
      ",nome_ponto_inicio    as start_point_name " \
      ",ponto_final          as end_point "\
      ",seq_fim              as sequence_number_end "\
      ",nome_ponto_final     as end_point "\
      ",nome_linha           as end_point_name "\
"from rota_sequenciada "\
"where "\
        "sourcedate ='2019-03-14' "\
    "and cod_linha in ('670','671','673','674','701','703','706','760','761','762','776','777','778') "\
"order by cod_linha,sentido_linha,cast(seq_inicio as int)"

In [23]:
rotas_df = sqlContext.sql(query_rotas).toPandas()

In [26]:
rotas_df.head()

,line_code,line_direction,start_point,sequence_number_start,start_point_name,end_point,sequence_number_end,end_point,end_point_name
0,670,Praca Rui Barbosa,180832,1,"Rua Angelina Ancai, 2 - Fazendinha",180831,2,"Rua Edvino Antonio Deboni, 15 - Fazendinha",S?O JORGE
1,670,Praca Rui Barbosa,180831,2,"Rua Edvino Antonio Deboni, 15 - Fazendinha",180829,3,"Rua Professor Elevir Dionysio, 415 - Fazendinha",S?O JORGE
2,670,Praca Rui Barbosa,180829,3,"Rua Professor Elevir Dionysio, 415 - Fazendinha",180827,4,"Rua Prof. Elevir Dionysio, 243 - Fazendinha",S?O JORGE
3,670,Praca Rui Barbosa,180827,4,"Rua Prof. Elevir Dionysio, 243 - Fazendinha",180048,5,"R. Carlos Klemtz, 505 - Fazendinha",S?O JORGE
4,670,Praca Rui Barbosa,180048,5,"R. Carlos Klemtz, 505 - Fazendinha",180697,6,"Rua Desembargador Lauro Sodre Lopes, 486 - Port?o",S?O JORGE


In [27]:
rotas_df.to_csv('sequenced_routes.csv',index=False)

In [28]:
import findspark
findspark.init()
import pyspark
import random
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import input_file_name

class ETLSpark:

    def __init__(self):
        self.conf  = SparkConf().setAppName("App")
        self.conf = (self.conf.setMaster('local[*]')
        .set('spark.executor.memory', '4G')
        .set('spark.driver.memory', '30G')
        .set('spark.driver.maxResultSize', '10G'))

        self.sc = SparkContext.getOrCreate(conf=self.conf)
        self.sqlContext = SQLContext(self.sc)

    def extract(self,src):
        return self.sqlContext.read.json(src).withColumn("filename", input_file_name())

    def transform(self, src_data, target_path, coalesce=1):
        src_data.coalesce(coalesce).write.mode('overwrite').format("parquet").save(target_path)


source_path = '/home/altieris/datascience/data/curitibaurbs/raw/linhas'
target_path = '/home/altieris/datascience/data/curitibaurbs/processed/linhas/'

etlspark = ETLSpark()
raw_data = etlspark.extract(source_path)
etlspark.transform(raw_data, target_path)

In [21]:
lines_raw = extract(source_path)
transform(lines_raw, target_path)

In [26]:
sqlContext.read.parquet(target_path).show()

+-----------------+---+--------------------+--------+--------------+--------------------+
|CATEGORIA_SERVICO|COD|                NOME|NOME_COR|SOMENTE_CARTAO|            filename|
+-----------------+---+--------------------+--------+--------------+--------------------+
|     CONVENCIONAL|464|A. MUNHOZ / J. BO...| AMARELA|             S|file:///home/alti...|
|      ALIMENTADOR|226|              ABAETE| LARANJA|             N|file:///home/alti...|
|          TRONCAL|182|           ABRANCHES| AMARELA|             N|file:///home/alti...|
|      ALIMENTADOR|332|            ACROPOLE| LARANJA|             N|file:///home/alti...|
|      ALIMENTADOR|334|            AGRICOLA| LARANJA|             N|file:///home/alti...|
|     CONVENCIONAL|265|   AHU / LOS ANGELES| AMARELA|             S|file:///home/alti...|
|     CONVENCIONAL|560|        ALFERES POLI| AMARELA|             S|file:///home/alti...|
|      ALIMENTADOR|232|             ALIANCA| LARANJA|             N|file:///home/alti...|
|      ALI